# Start running here

In [1]:
# time to structure a function that can do all of these at one go

def extract_df_pre2(link):
    companyName = []
    CIK = []
    fileDate = []
    ek_link = []
    
    base_url = "http://www.sec.gov"
    ek_url = base_url + link

    page_response_p = requests.get(ek_url, timeout=5)
    page_content_p = BeautifulSoup(page_response_p.content, "html.parser")
    
    split_name_cik = page_content_p.find_all('span', attrs = {'class': 'companyName'})[0].text.split(' (Filer)')
    
    companyName.append(split_name_cik[0])
    CIK.append(int(re.search(r'\d+', split_name_cik[1]).group()))
    fileDate.append(page_content_p.find_all('div', attrs = {'class': 'info'})[0].text)
    
    tr_content = page_content_p.find_all('tr')
    for items in tr_content[1:]:
        all_td = items.find_all('td')
        for i in range(len(all_td)):
            if all_td[i].text == '8-K':
                ek_link.append(all_td[i-1].a['href'])
                
    df = pd.DataFrame({"companyName": companyName, "CIK": CIK, "filedDate": fileDate, "url": ek_link})
    
    return df

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

twoM_data = pd.read_csv("pre_links.csv", index_col=None)

twoM_data.head()

,links
0,/Archives/edgar/data/1561092/00014931521400228...
1,/Archives/edgar/data/1561092/00014931521400000...
2,/Archives/edgar/data/1561092/00014931521300274...
3,/Archives/edgar/data/1561092/00014931521300105...
4,/Archives/edgar/data/1561092/00014931521300058...


In [3]:
twoM_0_2k = twoM_data[1500000:1599999]

print(len(twoM_0_2k))
twoM_0_2k.head()

99999


,links
1500000,/Archives/edgar/data/1039828/00010398281700008...
1500001,/Archives/edgar/data/1039828/00010398281700007...
1500002,/Archives/edgar/data/1039828/00011046591703990...
1500003,/Archives/edgar/data/1039828/00011046591703897...
1500004,/Archives/edgar/data/1039828/00011046591703882...


In [4]:
with ThreadPoolExecutor(max_workers=None) as executor:
    start = time.time()
    extract_df_results = []
    futures = [ executor.submit(extract_df_pre2, url) for url in twoM_0_2k['links']]
    for result in as_completed(futures):
        extract_df_results.append(result)
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))

Time Taken: 1119.416787s


In [4]:
extracted_df_results = []
for url in twoM_0_2k["links"]:
    extracted_df_results.append(extract_df_pre2(url))

TypeError: 'NoneType' object is not subscriptable

In [5]:
extract_df_results[0].result()

TypeError: 'NoneType' object is not subscriptable

In [6]:
df_results = []

for i in extract_df_results:
    try:
        df_results.append(i.result())
    except:
        pass
    
combined = pd.concat(df_results)

combined

,companyName,CIK,filedDate,url
0,AMERICAN EQUITY INVESTMENT LIFE HOLDING CO,1039828,2016-06-08,/Archives/edgar/data/1039828/00010398281600022...
0,AMERICAN EQUITY INVESTMENT LIFE HOLDING CO,1039828,2016-07-01,/Archives/edgar/data/1039828/00010398281600023...
0,AMERICAN EQUITY INVESTMENT LIFE HOLDING CO,1039828,2016-10-03,/Archives/edgar/data/1039828/00013410041600166...
0,AMERICAN EQUITY INVESTMENT LIFE HOLDING CO,1039828,2016-06-16,/Archives/edgar/data/1039828/00010398281600022...
0,AMERICAN EQUITY INVESTMENT LIFE HOLDING CO,1039828,2016-08-01,/Archives/edgar/data/1039828/00010398281600023...
...,...,...,...,...
0,Symetra Financial CORP,1403385,2014-01-30,/Archives/edgar/data/1403385/00014033851400000...
0,Symetra Financial CORP,1403385,2015-10-28,/Archives/edgar/data/1403385/00009501571500125...
0,Symetra Financial CORP,1403385,2013-05-07,/Archives/edgar/data/1403385/00012999331300084...
0,Symetra Financial CORP,1403385,2013-04-24,/Archives/edgar/data/1403385/00011931251317044...


In [7]:
combined.to_csv("first2mil-remain.csv")

In [1]:
import pandas as pd
df_1 = pd.read_csv("first2mil-remain.csv")
df_2 = pd.read_csv("first1-200m.csv")
df_3 = pd.read_csv("first12-400m.csv")
df_4 = pd.read_csv("first14-600m.csv")
df_5 = pd.read_csv("first16-800m.csv")
df_6 = pd.read_csv("first18-200m.csv")

In [4]:
list_df = [df_1,df_2,df_3,df_4,df_5,df_6]
df_new = pd.concat(list_df)
df_new.head()

,Unnamed: 0,companyName,CIK,filedDate,url
0,0,VESTIN GROUP INC,1068132,2002-04-25,/Archives/edgar/data/1068132/00009501500200037...
1,0,VESTIN GROUP INC,1068132,2005-03-28,/Archives/edgar/data/1068132/00009501290500287...
2,0,SUNDERLAND ACQUISITION CORP,1068132,1999-05-04,/Archives/edgar/data/1068132/
3,0,SUNDERLAND CORP,1068132,1999-12-17,/Archives/edgar/data/1068132/000091205799009827/
4,0,VESTIN GROUP INC,1068132,2003-08-08,/Archives/edgar/data/1068132/00011575230300388...


In [5]:
df_new.to_csv("1mill to end.csv")

## The Extraction code starts here

In [3]:
import pandas as pd
df1 = pd.read_csv(r"Data\0-100k.csv")
df2 = pd.read_csv(r"Data\100-199k.csv")
df3 = pd.read_csv(r"Data\200-399k.csv")
df4 = pd.read_csv(r"Data\400-499k.csv")
df5 = pd.read_csv(r"Data\500-599k.csv")
df6 = pd.read_csv(r"Data\600-699k.csv")
df7 = pd.read_csv(r"Data\700-799k.csv")
df8 = pd.read_csv(r"Data\800-899k.csv")
df9 = pd.read_csv(r"Data\1100-1199k.csv")
df10 = pd.read_csv(r"Data\1200-1299k.csv")
df11 = pd.read_csv(r"Data\1300-1399k.csv")
df12 = pd.read_csv(r"Data\1400-1499k.csv")
df13 = pd.read_csv(r"Data\1500-1599k.csv")
df14 = pd.read_csv(r"Data\1600-1699k.csv")
df15 = pd.read_csv(r"Data\1700-1799k.csv")
df16 = pd.read_csv(r"Data\1800-1899k.csv")
df17 = pd.read_csv(r"Data\1900-1999k.csv")

In [4]:
df_final = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17])

In [6]:
print(len(df_final))
df_final.head()

1184024


,companyName,CIK,filedDate,url
0,"Avangard Capital Group, Inc",1561092,2013-04-02,/Archives/edgar/data/1561092/00014931521300058...
1,"Avangard Capital Group, Inc",1561092,2013-04-02,/Archives/edgar/data/1561092/00014931521300058...
2,AIRLEASE LTD,799033,2004-01-29,/Archives/edgar/data/799033/000109230604000050...
3,AIRLEASE LTD,799033,2002-08-28,/Archives/edgar/data/799033/000109230602000291...
4,AIRLEASE LTD,799033,2004-09-30,/Archives/edgar/data/799033/000109230604000729...


In [7]:
df_final.to_csv("compiled_urls.csv")

In [46]:
link = df_final["url"].iloc[0]

In [47]:
base_url = "http://www.sec.gov"
ek_url = base_url + link

In [48]:
ek_url
#str(ek_url).split(",")

'http://www.sec.gov/Archives/edgar/data/1561092/000149315213000582/form8k.htm'

In [50]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

## Managed to get the text but the page has to be 8k page

In [75]:
page_response_p = requests.get(ek_url, timeout=5)
page_content_p = BeautifulSoup(page_response_p.content, "html.parser")

split_name_cik = page_content_p.find_all('font', attrs = {'style': 'font: 10pt Times New Roman, Times, Serif'})#.text.split(' (Filer)')
split_name_cik[45].text

'On\nMarch 26, 2013, Avangard Capital Group, Inc.(“we”, “us”, or “our”) entered into an Retail\nInstallment Contract Receivable Purchase Agreement (the “Purchase Agreement”) with Avangard Auto Finance, Inc. and\nAvangard Financial Group, Inc. (the “Sellers”), both of whom are our affiliates. Under the terms of the Purchase Agreement,\nwe agreed to pay the Sellers $102,250 (the “Purchase Price”) to purchase certain of their retail installment contract\nreceivables as of the date of the agreement. These amounts include an aggregate principal balance of approximately $141,868for\ncurrent loans receivables (the “Current Loans”) and an aggregate principal balance of approximately $323,449 for non-current\nloans receivables (the “Non-Current Loans”). The Purchase Price has been allocated as follows: $100,000 for the Current\nLoans which we will pay upon execution of the agreement and $2,250 has been allocated to the Non-Current Loans payable which we\nwill pay within thirty (30) days of the 

In [80]:
def extract_final():
    info = []
    for i in df_final["url"].values:
        base_url = "http://www.sec.gov"
        ek_url = base_url + i 
        page_response_p = requests.get(ek_url, timeout=5)
        page_content_p = BeautifulSoup(page_response_p.content, "html.parser")

        split_name_cik = page_content_p.find_all('font', attrs = {'style': 'font: 10pt Times New Roman, Times, Serif'})#.text.split(' (Filer)')
        para = split_name_cik[45].text
        info.append(para)
    return info

## Stuck, not the same page for all urls

In [81]:
info = extract_final()

IndexError: list index out of range